In [ ]:
from requests_html import HTMLSession # for making request
import requests # for making request
import json # for API parsing
import pandas as pd # for data processing
import numpy as np # for data processing
from tqdm import tqdm # for count time of iteration
import re
from tqdm import tqdm
from imdb import IMDb
from imdb.Person import Person

# Crawl user data

Vì IDBm không có API để lấy thông tin user, nên phần này mình hoàn toàn dựa vào parse HTML để làm.
Mỗi bộ phim mình sẽ lần lượt lấy top profilic user đánh giá từ 1 đến 10 sao. Lí do chọn profilic user vì những user này đánh giá nhiều phim, mình sẽ có tỉ lệ cao tìm được user đó đánh giá những phim khác trong data movie. Với khoảng 9000 bộ phim, mình crawl được tầm hơn 1 triệu bộ (movie_id, user_id, rating)

# Load set id

In [4]:
movie_df = pd.read_csv('../data/movie.csv', dtype={'id':str})

In [ ]:
set_id = set(movie_df['id'])

# Parse HTML

In [ ]:
session = HTMLSession()

In [ ]:
users = {}
for id in tqdm(set_id_small):
    try:
        user_rating = {}
        for i in range(1,11,2):
            url = f'https://www.imdb.com/title/tt{id}/reviews?sort=reviewVolume&dir=desc&ratingFilter={i}'
            r = session.get(url)
            try:
                user = r.html.find('.display-name-link > a')
                user_rating[i] = user
            except:
                pass
        users[id] = user_rating
    except:
        pass

# Save data

In [ ]:
def get_id_from_element(ele):
    id = re.search('/ur(.+?)/', ele.links.pop())
    if id is None:
        return None
    return id.group(1)

In [ ]:
ratings = []
for key_id in tqdm(users):
    for key_rating in users[key_id]:
        for element in users[key_id][key_rating]:
            dict_row = {}
            user_id = get_id_from_element(element)
            dict_row['movie_id'] = key_id
            dict_row['user_id'] = user_id
            dict_row['rating'] = int(key_rating)
            ratings.append(dict_row)

In [ ]:
ratings_df = pd.DataFrame(data=ratings)

In [7]:
ratings_df.to_csv('../data/rating.csv', index=False)